# Deep Neural Net  (MNIST)
---
## initialize

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home='.')

In [2]:
N=10000 #reduce size of the dataset (70k)
indices = np.random.permutation(range(len(mnist.data)))[:N]
x = mnist.data[indices]
t = mnist.target[indices]
t = np.eye(10)[t.astype(int)]
X_train, X_test, Y_train, Y_test = train_test_split(x, t, test_size=0.2)

## basic tensorflow

In [3]:
import tensorflow as tf

class DNN:
    def __init__(self, n_in, n_hids, n_out):
        self.n_in, self.n_hids, self.n_out = n_in, n_hids, n_out
        self.weights, self.biases = [], []
        self._x, self_t = None, None
        self._keep_prob = None
        self._sess = None
        self._history = {'accuracy':[], 'loss':[]}
        self.Accuracy = None
        
    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    
    def bias_variable(self, shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)
    
    def inference(self, x, keep_prob):
        for i, n_hid in enumerate(self.n_hids):
            if i is 0:
                input = x
                input_dim = self.n_in
            else:
                input = output
                input_dim = self.n_hids[i-1]
                
            self.weights.append(self.weight_variable([input_dim, n_hid]))
            self.biases.append(self.bias_variable([n_hid]))
            
            h = tf.nn.relu(tf.matmul(input, self.weights[-1]) + self.biases[-1])
            output = tf.nn.dropout(h, keep_prob)
            
        self.weights.append(self.weight_variable([self.n_hids[-1], self.n_out]))
        self.biases.append(self.bias_variable([self.n_out]))
        
        return tf.nn.softmax(tf.matmul(output, self.weights[-1]) + self.biases[-1])
    
    def loss(self, y, t):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), axis=1))
        return cross_entropy
    
    def training(self, loss):
        return tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    def accuracy(self, y, t):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
        return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def fit(self, X_train, Y_train, epochs=100, batch_size=100, p_keep=0.5, verbose=1):
        x = tf.placeholder(tf.float32, shape=[None, self.n_in]) 
        t = tf.placeholder(tf.float32, shape=[None, self.n_out]) 
        keep_prob = tf.placeholder(tf.float32)
        
        self._x = x
        self._t = t
        self._keep_prob = keep_prob
        
        y = self.inference(x, keep_prob)
        loss = self.loss(y, t)
        train_step = self.training(loss)
        self.Accuracy = self.accuracy(y, t)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        self._sess = sess
        
        N_train = len(X_train)
        n_batches = N_train // batch_size
        
        for epoch in range(epochs):
            X_, Y_ = shuffle(X_train, Y_train)
            
            for i in range(n_batches):
                start = i * batch_size
                end = start +batch_size
                sess.run(train_step, feed_dict={x:X_[start:end], t:Y_[start:end], keep_prob:p_keep})
                loss_ = loss.eval(session=sess, feed_dict={x:X_train, t:Y_train, keep_prob:1.0})
                accuracy_ = self.Accuracy.eval(session=sess, feed_dict={x:X_train, t:Y_train, keep_prob:1.0})
                
                self._history['loss'].append(loss_)
                self._history['accuracy'].append(accuracy_)
                
            if verbose:
                print('epoch:', epoch, 'loss', round(loss_, 3), 'accuracy:', round(accuracy_, 3))
            
        return self._history
    
    def evaluate(self, X_test, Y_test):
        return self.Accuracy.eval(session= self._sess, feed_dict={self._x:X_test, self._t:Y_test, self._keep_prob:1.0})

In [ ]:
model = DNN(n_in=784, n_hids=[200,200], n_out=10)
model.fit(X_train, Y_train, epochs=10, batch_size=200, p_keep=0.5)

print('accuracy:', model.evaluate(X_test, Y_test))

epoch: 0 loss 0.543 accuracy: 0.827
epoch: 1 loss 0.441 accuracy: 0.852
